PROJECT TITLE: DEPRESSION RISK CLASSIFICATION USING MACHINE LEARNING.

GOAL: To build a machine learning model that automatically classifies depression severity(Minimal, Mild, Moderate) based on patient responses to the PHQ-9 questionnaire.

DATASET: PHQ-9 Dataset from Mendeley

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#loading the dataset
df =pd.read_csv(r"C:\Users\ADMIN\Downloads\PHQ-9_Dataset_5th Edition.csv")


# Clean Headers
df.columns = df.columns.str.strip()

# We group columns by their name and keep only the first instance.
df = df.groupby(level=0, axis=1).first()

# 4. Define Questions
phq_questions = [
    'Little interest or pleasure in doing things',
    'Feeling down, depressed, or hopeless',
    'Trouble falling or staying asleep, or sleeping too much',
    'Feeling tired or having little energy',
    'Poor appetite or overeating',
    'Feeling bad about yourself—or that you are a failure or have let yourself or your family down',
    'Trouble concentrating on things, such as reading the newspaper or watching television',
    'Moving or speaking so slowly that other people could have noticed? Or the opposite—being so fidgety or restless that you have been moving around a lot more than usual',
    'Thoughts that you would be better off dead or of hurting yourself in some way'
]

# 5. Define Mapping
phq_mapping = {
    'Not at all': 0,
    'Several days': 1,
    'More than half the days': 2,
    'Nearly every day': 3
}

# Apply Cleaning & Mapping
for col in phq_questions:
    # Check if column exists to avoid KeyError
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().map(phq_mapping)

print("Success! Missing values in questions:", df[phq_questions].isna().sum().sum())
print(df[phq_questions].head())


Success! Missing values in questions: 0
   Little interest or pleasure in doing things  \
0                                            2   
1                                            0   
2                                            0   
3                                            3   
4                                            0   

   Feeling down, depressed, or hopeless  \
0                                     0   
1                                     0   
2                                     0   
3                                     3   
4                                     0   

   Trouble falling or staying asleep, or sleeping too much  \
0                                                  0         
1                                                  3         
2                                                  0         
3                                                  0         
4                                                  0         

   Feeling tired or having 

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5996\1556778558.py:18: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(level=0, axis=1).first()


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 16 columns):
 #   Column                                                                                                                                                                  Non-Null Count  Dtype 
---  ------                                                                                                                                                                  --------------  ----- 
 0   Age                                                                                                                                                                     682 non-null    int64 
 1   Feeling bad about yourself—or that you are a failure or have let yourself or your family down                                                                           682 non-null    int64 
 2   Feeling down, depressed, or hopeless                                                                      

From the dataset above, we can see that it contains 682 individuals assessed using the PHQ-9 depression screening questionnaire along with contextual factors such as age, gender, sleep quality, study pressure, financial pressure. 

Also, each response is categorical, i.e, not at all, several days, more than half days and nearly every day. These are ordinal variables, not nominal.


We will now Prepare the Target(PHQ_Severity)

Our target variable is categorical(e.g. severe, mild). We map these categories to an ordered scale of: 

 0: None-Minimal

 1: Mild

 2: Moderate

 3: Moderately Severe

 4: Severe

In [5]:
#checking unique features to ensure we map correctly
print("Unique Severity Labels:", df['PHQ_severity'].unique())

#Defining Target Mapping(ordinal)
severity_mapping = {
    'Minimal': 0,
    'Mild': 1,
    'Moderate': 2,
    'Moderately severe': 3,
    'Severe': 4,
    'None-minimal': 0 # Handling potential variation in data
}

# 3. Apply Mapping
df['Severity_Encoded'] = df['PHQ_Severity'].str.strip().map(severity_mapping)

# 4. Drop rows where target might be missing
df = df.dropna(subset=['Severity_Encoded'])

print("Target distribution:")
print(df['Severity_Encoded'].value_counts().sort_index())

KeyError: 'PHQ_severity'

We now want to define the 9 PHQ questions(features)

We will now define the mapping: We will map them in the following manner:

# Response                Value

Not at all                0

Several days              1

More than half the days   2

Nearly every day          3

We are now encoding other categorical variables:

We now want to do ordinal mappings

In [12]:
ordinal_mappings = {
    'Sleep Quality': {'Worst': 0, 'Average': 1, 'Good': 2},
    'Study Pressure': {'Bad': 0, 'Average': 1, 'Good': 2},
    'Financial Pressure': {'Bad': 0, 'Average': 1, 'Good': 2}
}
# Applying them
for col, mapping in ordinal_mappings.items():
    df[col] = df[col].map(mapping)


We now want to do Feature-Target Split

In [13]:
X = df.drop(columns=['HighRisk_Depression'])

y = df['HighRisk_Depression']

print(X.shape,y.shape)

(682, 14) (682,)


We will now perform Stratified Train-Test Split. The reason why we are using stratified sampling is to preserve class distribution

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size =0.25, stratify= y,random_state=42
)
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

HighRisk_Depression
0    0.528376
1    0.471624
Name: proportion, dtype: float64
HighRisk_Depression
0    0.532164
1    0.467836
Name: proportion, dtype: float64


From the results above, we can see that the dataset ehibits a slight class imbalance with approximately 53% low risk and 47% high risk cases. Our baseline accuracy is 53% and any model we will build must outperform this to be meaningful. The reason why this baseline is critical is that it can be used for model comparison and demonstrating improvement

The ordinal variables were encoded using domain-informed mappings and nominal variables were one-hot encoded.

In [15]:
#One-Hot Encoding nominal columns
nominal_cols = ['Gender']

X_train = pd.get_dummies(X_train, columns=nominal_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=nominal_cols, drop_first=True)

# Align columns
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

We will do Feature Matrix to exclude labels and text columns

In [16]:
#Feature Matrix
X = df.drop(columns=[
    'HighRisk_Depression'
])


In [17]:
# Check how many non-null values are in these columns
print(df[phq_columns].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 9 columns):
 #   Column                                                                                                                                                                  Non-Null Count  Dtype
---  ------                                                                                                                                                                  --------------  -----
 0   Little interest or pleasure in doing things                                                                                                                             682 non-null    int64
 1   Feeling down, depressed, or hopeless                                                                                                                                    682 non-null    int64
 2   Trouble falling or staying asleep, or sleeping too much                                                        

We will now handle the missing values(NaNs) in our training data and we will use a SimpleImputer

In [18]:
from sklearn.impute import SimpleImputer

# 1. Initialize the Imputer (using median is best for ordinal health data)
imputer = SimpleImputer(strategy='median')

# 2. Fit and transform the training data, and transform the test data
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

print("Missing values handled.")

Missing values handled.


We now need to do Feature scalling because PHQ Scores, age, pressure variables are on different scales and also it prevents models from being biased towards large-value features.

In [19]:
#The next step is doing numeric feature scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# we will use the imputed here
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)



We now want to do Logistic Regression and it will be our Benchmark Model

In [20]:
from sklearn.linear_model import LogisticRegression

# Initializing with 'balanced' weights to handle the slight class imbalance
log_reg = LogisticRegression(class_weight='balanced', random_state=42)


log_reg.fit(X_train_scaled, y_train)
print("Logistic Regression Model Fitted Successfully!")

Logistic Regression Model Fitted Successfully!


We will now do model evaluation

In [21]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = log_reg.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))


Accuracy: 1.0

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        91
           1       1.00      1.00      1.00        80

    accuracy                           1.00       171
   macro avg       1.00      1.00      1.00       171
weighted avg       1.00      1.00      1.00       171

Confusion Matrix:

[[91  0]
 [ 0 80]]


Tree-Based Model(Random Forest)

We are doing the Random forest because it handles non-linear relationships, it is robust to noise and works well with mixed feature types. It also captures symptom interactions.

In [22]:
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier(
    n_estimators = 200,
    class_weight = 'balanced',
    random_state = 42
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98        91
           1       0.98      0.99      0.98        80

    accuracy                           0.98       171
   macro avg       0.98      0.98      0.98       171
weighted avg       0.98      0.98      0.98       171

[[89  2]
 [ 1 79]]
